In [420]:
import csv 
import re
import pandas as pd
import fitz
import os

# TO DO LIST for clean (each individual)
- Page splits
- Same prof, different locations (multiple new lines)

In [421]:
def extract_text_from_pdf(pdf_path):
    # Open the provided PDF file
    doc = fitz.open(pdf_path)
    text = ""
    
    # Extract text from each page
    for page in doc:
        text += page.get_text()
    
    doc.close()
    return text

In [422]:
# class replacement lines
name_list = [
    'COMPUTER\nSCIENCE\nFOR\nSTEM',
    'INTRODUCTION\nTO\nCOMPUTER\nSCIENCE',
    'DATA\nSTRUCTURES\nAND\nALGORITHMS',
    'FOUNDATIONAL\nPROGRAMMING\nCONCEPTS &\nDESIGN'
    'DATA\nSTRUCTURES\nAND\nALGORITHMS', 
    'PYTHON-BASED\nCOMPUTING:\nBUILDING A\nSENSOR SYSTEM', 
    'INTRODUCTION\nTO THE LINUX\nOPERATING\nSYSTEM', 
    'SOFTWARE\nENGINEERING', 
    'COMPUTER\nORGANIZATION', 
    'DISCRETE\nMATHEMATICS',
    'ADVANCED\nSOFTWARE\nENGINEERING',
    'PRINCIPLES OF\nPROGRAMMING\nLANGUAGES', 
    'DATA BASE\nMANAGEMENT', 
    'ALGORITHMS', 
    'GAME THEORY\nAND NETWORKS',
    'INFORMATION\nSECURITY AND\nPRIVACY', 
    'GRADUATE\nTHESIS /\nDISSERTATION\nRESEARCH\nCREDIT',
    'INTRODUCTION\nTO\nBIOINFORMATICS',
    'COMPOTER\nSCIENCE\nHONORS THESIS',
    'MASTERS\nPROJECT\nCREDITS',
    'HUMAN-ROBOT\nINTERACTION',
    'OPERATING\nSYSTEMS',

    'COMPUTER\nNETWORKS I', 
    'COMPUTER\nNETWORKS II',

    'PROGRAMMING\nCHALLENGES I', 
    'PROGRAMMING\nCHALLENGES II',
    'PROGRAMMING\nCHALLENGES III',

    'FOUNDATIONAL\nPROGRAMMING CONCEPTS &\nDESIGN',
    'CS@MINES\nBRIDGE\nSEMINAR\nCOURSE',
    
    'COMPUTER\nGRAPHICS', 
    'INTRODUCTION\nTO COMPUTER\nVISION',
    'INTRODUCTION\nTO MACHINE\nLEARNING',
    'FORTRAN\nPROGRAMMING',
    'INDUSTRY\nEXPLORATION I', 
    'INDUSTRY\nEXPLORATION II',
    'INDUSTRY\nEXPLORATION III',
    'WEB\nPROGRAMMING', 
    'WEB\nAPPLICATIONS',
    'INTRODUCTION\nTO DATA\nSCIENCE',
    'COMPUTER\nSIMULATION',
    

    'INTRODUCTION\nTO COMPUTER\nSCIENCE - LAB',
    'INTRODUCTION\nTO COMPUTER\nSCIENCE',
    'PROGRAMMING\nCONCEPTS', 
    'DATA\nSTRUCTURES',
    'COMPUTER\nSCIENCE FOR\nSTEM',

    'PYTHON-BASED\nCOMPUTING:\nBUILDING A\nSENSOR\nSYSTEM', 

    'INTRODUCTION\nTO\nCRYPTOGRAPHY',
    'ELEMENTS OF\nGAMES AND\nGAME\nDEVELOPMENT',
    'ELEMENTS OF\nCOMPUTING\nSYSTEMS',
    'COMPILER\nDESIGN',
    'HUMAN-\nCENTERED\nROBOTICS',

    'MOBILE\nAPPLICATION\nDEVELOPMENT',
]

# Extract text from PDFs

In [423]:
# find all files with .pdf extension in directory schedules
pdf_files = [f for f in os.listdir('../schedules') if f.endswith('.pdf')]

In [424]:
# extract text from pdf files
for pdf_path in pdf_files: 
    text = extract_text_from_pdf("../schedules/" + pdf_path)
    # save text
    with open("../schedules/raw_" + pdf_path.replace(".pdf", "") + '.txt', 'w') as f:

        # strip extra spaces from end of lines
        text = re.sub(r'\s+$', '', text, flags=re.MULTILINE)

        # remove header details
        header_end_marker = 'Location Attribute'
        end_column_location = text.index(header_end_marker)
        text = text[end_column_location + len(header_end_marker) + 1:]

        # renmove footer details
        footer_marker = 'New Search\n© 2024 Ellucian Company L.P. and its affiliates.'
        text = text.replace(footer_marker, '')  

        # replace UG schld with UG_Req 
        ug_req_schld_flag = 'UG\nRequired\nCrse for\nSchdlng'
        ta_ra_flag = 'Summer\nGraduate\nTA/RA Fee'

        text = text.replace(ug_req_schld_flag, 'UG_Req\n').replace(ta_ra_flag, 'Summer_Grad_TARA_Fee\n')

        # replace ONLINE NR tag
        text = text.replace('ONLINE NR', 'ONLINE')
        text = text.replace('ONLINE UG_Req\n', 'ONLINE\nUG_Req\n')
        for code in ["NR", "C"]:
            text = text.replace(f"ONLINE\n{code}", f"ONLINE\n\n{code}")
            text = text.replace(f"UG_Req\n{code}", f"UG_Req\n\n{code}")

        # replace TBA with false scheduling information
        text = text.replace('\nTBA','\nAAA\n00:00\nzm-\n00:00\nzm')

        # replace class names
        for name in name_list:
            text = text.replace(name, name.replace('\n', ' '))

        f.write(text)

In [425]:
SEMESTER_ID = 'Spring24'

In [426]:
# open file to read 
with open(f"../schedules/clean_{SEMESTER_ID}.txt", 'r') as f:
    text = f.read()

In [427]:
# replace block endings
pattern = r'([A-Z]{2} \d{3})\n(NR|C)\n(\d{5})'
nr_pattern = r'(NR NR)\n(NR|C)\n(\d{5})'
replacement = r'\1\n\n\2\n\3'

text = re.sub(pattern, replacement, text)
text = re.sub(nr_pattern, replacement, text)

In [428]:
block_list = text.split('\n\n')
print(block_list)

['C\n12145 CSCI\n128\nB\nC\n3.000\nCOMPUTER SCIENCE FOR STEM\nMWF\n12:00\npm-\n12:50\npm\n63\n63\n0\n0\n0\n0\n0\n0\n0\nDrew Robert\nCornmesser\n(P), Robert\nHaven\nThompson\n01/09-\n05/10\nBB W210\nUG_Req', 'NR\n12146 CSCI\n128\nC\nC\n3.000\nCOMPUTER SCIENCE FOR STEM\nMWF\n12:00\npm-\n12:50\npm\n80\n78\n2\n0\n0\n0\n0\n0\n0\nChristine\nLiebe (P),\nRobert Haven\nThompson\n01/09-\n05/10\nGC MET\nUG_Req', 'NR\n12147 CSCI\n128\nD\nC\n3.000\nCOMPUTER SCIENCE FOR STEM\nMWF\n01:00\npm-\n01:50\npm\n60\n58\n2\n0\n0\n0\n0\n0\n0\nRobert Haven\nThompson\n(P)\n01/09-\n05/10\nMZ 022\nUG_Req', 'NR\n12148 CSCI\n128\nE\nC\n3.000\nCOMPUTER SCIENCE FOR STEM\nMWF\n01:00\npm-\n01:50\npm\n60\n59\n1\n0\n0\n0\n0\n0\n0\nChristine\nLiebe (P),\nRobert Haven\nThompson\n01/09-\n05/10\nBB W210\nUG_Req', 'NR\n12149 CSCI\n128\nF\nC\n3.000\nCOMPUTER SCIENCE FOR STEM\nMWF\n01:00\npm-\n01:50\npm\n60\n58\n2\n0\n0\n0\n0\n0\n0\nUmut Mete\nSaka (P),\nRobert Haven\nThompson\n01/09-\n05/10\nGC MET\nUG_Req', 'NR\n12150 CSCI\n12

In [429]:
split_list = []
for index,block in enumerate(block_list): 
    container = []
    for line in block.split('\n'):
        if line != '':
            container.append(line)
    print(f"LINE #{index}")
    
    if container != []:
        try: 
            # add end flag (for counts)
            if container[-1] != 'UG_Req' and container[-1] != 'Summer_Grad_TARA_Fee' and container[-1] != 'Elective':
                container.append('No_Special_Notes')

            # remove start flag (for counts)
            if container[0] != 'C' and container[0] != 'NR': 
                raise KeyError
            
            # combine credit ranges
            if container[5][-1] == '-':
                container[5] = container[5] + container[6]
                container.remove(container[6])

            # combine dates
            container[-4] = container[-4] + container[-3]
            container.remove(container[-3])

            # combine times
            container[8] = container[8] + container[9] + container[10] + container[11]
            for i in range(11,8,-1):
                container.remove(container[i])

            # combine instructors
            container[18] = " ".join([container[i].replace(" ,", ",") for i in range(18, len(container) - 3)])
            for i in range(len(container) - 4, 18, -1):
                container.remove(container[i])

            # split CRN from Crse #
            l = container[1].split(" ") 
            container[1] = l[0]
            container.insert(2, l[1])

            # add to container
            split_list.append(container)

            print("\t", container)
            print("\n")

        except: 
            print("ERROR")
            print(block)
            print(container)

print(split_list)

LINE #0
	 ['C', '12145', 'CSCI', '128', 'B', 'C', '3.000', 'COMPUTER SCIENCE FOR STEM', 'MWF', '12:00pm-12:50pm', '63', '63', '0', '0', '0', '0', '0', '0', '0', 'Drew Robert Cornmesser (P), Robert Haven Thompson', '01/09-05/10', 'BB W210', 'UG_Req']


LINE #1
	 ['NR', '12146', 'CSCI', '128', 'C', 'C', '3.000', 'COMPUTER SCIENCE FOR STEM', 'MWF', '12:00pm-12:50pm', '80', '78', '2', '0', '0', '0', '0', '0', '0', 'Christine Liebe (P), Robert Haven Thompson', '01/09-05/10', 'GC MET', 'UG_Req']


LINE #2
	 ['NR', '12147', 'CSCI', '128', 'D', 'C', '3.000', 'COMPUTER SCIENCE FOR STEM', 'MWF', '01:00pm-01:50pm', '60', '58', '2', '0', '0', '0', '0', '0', '0', 'Robert Haven Thompson (P)', '01/09-05/10', 'MZ 022', 'UG_Req']


LINE #3
	 ['NR', '12148', 'CSCI', '128', 'E', 'C', '3.000', 'COMPUTER SCIENCE FOR STEM', 'MWF', '01:00pm-01:50pm', '60', '59', '1', '0', '0', '0', '0', '0', '0', 'Christine Liebe (P), Robert Haven Thompson', '01/09-05/10', 'BB W210', 'UG_Req']


LINE #4
	 ['NR', '12149', 'CS

In [430]:
# save to csv
with open(f'../schedules/{SEMESTER_ID}.csv', 'w') as f:
    writer = csv.writer(f)

    # write header
    writer.writerow(['Select', 'CRN', 'Subj', 'Section', 'CMP', 
                        'Credits', 'Title', 'Days', 'Time', 
                        'Cap', 'Act', 'WL Cap', 'WL Act', 'WL Rem', 'XL Cap', 'XL Act', 'XL Rem', 
                        'Instructors', 'Date', 'Location', 'Attributes'])
    writer.writerows(split_list)

In [431]:
# load csv into df
df = pd.read_csv(f'../schedules/{SEMESTER_ID}.csv')
display(df)

Select    CRN Subj Section  CMP  \
C  12145   CSCI    128    B       C  3.0   
NR 12146   CSCI    128    C       C  3.0   
   12147   CSCI    128    D       C  3.0   
   12148   CSCI    128    E       C  3.0   
   12149   CSCI    128    F       C  3.0   
   12150   CSCI    128    G       C  3.0   
   12151   CSCI    128    H       C  3.0   
   12152   CSCI    128    I       C  3.0   
   12153   CSCI    128    J       C  3.0   
   12154   CSCI    128    K       C  3.0   
C  11598   CSCI    200    A       C  3.0   
   11594   CSCI    200    B       C  3.0   
   11595   CSCI    200    C       C  3.0   
   11596   CSCI    200    D       C  3.0   
   11597   CSCI    200    E       C  3.0   
NR 11599   CSCI    220    A       C  3.0   
   11600   CSCI    220    B       C  3.0   
C  10732   CSCI    250    A       C  3.0   
   12238   CSCI    250    B       C  3.0   
NR 10711   CSCI    274    A       C  1.0   
   10393   CSCI    274    B       C  1.0   
   10758   CSCI    274    C       C  1.0   
   10733   CSCI    303    A       C  3.0   
   11184   CSCI    303    B       C  3.0   
C  11390   CSCI    306    A       C  3.0   
NR 10034   CSCI    306    B       C  3.0   
   11602   CSCI    306   OL       O  3.0   
C  10759   CSCI    341    A       C  3.0   
NR 10624   CSCI    358    A       C  3.0   
   11523   CSCI    358    B       C  3.0   
   11289   CSCI    400    A       C  3.0   
   10972   CSCI    403    A       C  3.0   
   10635   CSCI    403    B       C  3.0   
C  10734   CSCI    404    A       C  3.0   
NR 12141   CSCI    404    B       C  3.0   
C  10735   CSCI    406    A       C  3.0   
NR 12140   CSCI    406    B       C  3.0   
C  11273   CSCI    410    A       C  3.0   
NR 11386   CSCI    425    A       C  3.0   
   11166   CSCI    432    A       C  3.0   
C  11331   CSCI    442    A       C  3.0   
NR 10038   CSCI    442    B       C  3.0   
   11200   CSCI    442    C       C  3.0   
   12285   CSCI    444    A       C  3.0   
C  11383   CSCI    446    A       C  3.0   
   11893   CSCI    470   OL       O  3.0   
NR 10590   CSCI    474    A       C  3.0   
   10790   CSCI    477    A       C  3.0   
   12156   CSCI  498A     A       C  3.0   
   12157   CSCI  498B     A       C  3.0   
   12275   CSCI  498D     A       C  3.0   

                                                   Credits Title  \
C  12145                         COMPUTER SCIENCE FOR STEM   MWF   
NR 12146                         COMPUTER SCIENCE FOR STEM   MWF   
   12147                         COMPUTER SCIENCE FOR STEM   MWF   
   12148                         COMPUTER SCIENCE FOR STEM   MWF   
   12149                         COMPUTER SCIENCE FOR STEM   MWF   
   12150                         COMPUTER SCIENCE FOR STEM   MWF   
   12151                         COMPUTER SCIENCE FOR STEM   MWF   
   12152                         COMPUTER SCIENCE FOR STEM   MWF   
   12153                         COMPUTER SCIENCE FOR STEM   MWF   
   12154                         COMPUTER SCIENCE FOR STEM   MWF   
C  11598        FOUNDATIONAL PROGRAMMING CONCEPTS & DESIGN   MWF   
   11594        FOUNDATIONAL PROGRAMMING CONCEPTS & DESIGN   MWF   
   11595        FOUNDATIONAL PROGRAMMING CONCEPTS & DESIGN   MWF   
   11596        FOUNDATIONAL PROGRAMMING CONCEPTS & DESIGN   MWF   
   11597        FOUNDATIONAL PROGRAMMING CONCEPTS & DESIGN   MWF   
NR 11599                    DATA STRUCTURES AND ALGORITHMS   MWF   
   11600                    DATA STRUCTURES AND ALGORITHMS   MWF   
C  10732  PYTHON-BASED COMPUTING: BUILDING A SENSOR SYSTEM   MWF   
   12238  PYTHON-BASED COMPUTING: BUILDING A SENSOR SYSTEM   MWF   
NR 10711        INTRODUCTION TO THE LINUX OPERATING SYSTEM     M   
   10393        INTRODUCTION TO THE LINUX OPERATING SYSTEM     W   
   10758        INTRODUCTION TO THE LINUX OPERATING SYSTEM     F   
   10733                      INTRODUCTION TO DATA SCIENCE    TR   
   11184                      INTRODUCTION TO DATA SCIENCE    TR   
C  11390            